In [1]:
# from google.colab import drive
# drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import nltk
import contractions
import re
import math
import gensim
from gensim.test.utils import datapath
from gensim import utils
from gensim.models import Word2Vec
import gensim.downloader
from gensim.test.utils import datapath
from gensim import utils
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.metrics import accuracy_score


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [5]:
#df=pd.read_table('drive/My Drive/CSCI544/HW3/HW 3/amazon_reviews_us_Beauty_v1_00.tsv',on_bad_lines='skip',low_memory=False)

In [6]:
#df_fnl=df[['review_body','star_rating']]

In [7]:
#df_final=df_fnl.replace({'star_rating':{2:1,3:2,4:3,5:3,'5':3,'2':1,'3':2,'4':3,'1':1}})

In [8]:
#df_final.isnull().sum()

In [9]:
#df_final=df_final.dropna()

In [10]:
#df_final.isnull().sum()

In [11]:
# s0 = df_final[df_final['star_rating'].eq(1)].sample(20000).index
# s1 = df_final[df_final['star_rating'].eq(2)].sample(20000).index 
# s2 = df_final[df_final['star_rating'].eq(3)].sample(20000).index 

# df_fi = df_final.loc[s0.union(s1).union(s2)]

In [12]:
#def remove_alphanumeric(s):
#     s=s.lower()
#     s=s.strip()
#     s=contractions.fix(s)
#     s=s.replace(r'<[^<>]*>', '')
#     s=s.replace(r'http\S+', '').replace(r'www\S+', '')
#     s=re.sub(r'[^a-zA-Z]',' ',s)
#     return re.sub(' +', ' ',s)

In [13]:
#df_fi['review_body']=df_fi['review_body'].apply(remove_alphanumeric)

In [14]:
# path='drive/My Drive/CSCI544/HW3/HW 3/main_dataframe.csv'

# with open(path,'w',encoding='utf-8-sig') as f:
#   df_fi.to_csv(f)


In [3]:
#df_fi=pd.read_csv('drive/My Drive/CSCI544/HW3/HW 3/main_dataframe.csv')

df_fi=pd.read_csv('main_dataframe.csv')

**Word Embeddings**


Part a Pre-trained model


In [4]:
google_wrd2vc = gensim.downloader.load('word2vec-google-news-300')

In [5]:
# from gensim.models import KeyedVectors
# google_wrd2vc= KeyedVectors.load('google_wrd2vc.kv')

In [6]:
google_wrd2vc.similarity('good','better')

0.61207294

In [7]:
google_wrd2vc.similarity('machine','sounds')

0.16873193

Part B Own corpus

In [19]:
# class MyCorpus:
#     ## step to create custom Corpus for Word2vec to use to generate word vectors
#     def __iter__(self):
#         for line in df_fi['review_body']:
#             # assume there's one document per line, tokens separated by whitespace
#             yield utils.simple_preprocess(line)

In [20]:
# #Loading my own Word2Vec 
# my_model= Word2Vec(sentences = MyCorpus(), size = 300, window = 13, min_count = 9)


In [21]:
# my_model.wv.similarity('machine','sounds')

In [22]:
# my_model.wv.similarity('good','better')

In [23]:
# print(my_model.wv.most_similar(positive=['king', 'woman'], negative=['man'],topn=1))

The performance of the Pre-trained model by word2vec-google-news-300 is better since it has a much bigger corpus than my personal dataset. Therefore the pretrained mmodel can findout better similarities than my model.

**Simple Models**

In [24]:
# def wrd2vc_avg(strings, wv_model):
#   sent_split = strings.split(' ') if isinstance(strings, str) else strings
#   sum = np.zeros(shape=(300,))
#   count = 0
#   for word in sent_split:
#     if word in wv_model:
#       wv = wv_model[word]
#       sum += wv
#       count += 1
#   avg = sum / count
#   return avg

In [9]:
# def idx_nan(matrix):
#     idx = np.isnan(matrix).any(axis=1).nonzero()[0]
#     if len(idx) > 0:
#         return idx[::300]
#     else:
#         return None

def nan_id(mtrx):
    
      if np.any(np.isnan(mtrx)):
        nanarray=np.argwhere(np.isnan(mtrx))
        nannum=nanarray.shape[0]
        array=np.arange(0,nannum,300)

        id=[]
        for x in array:
            
            id.append(nanarray[x][0])
        return id
      else:
        return None

In [26]:
# true_labels= df_fi['star_rating'].to_numpy()
# X_train, X_test, Y_train, Y_test = train_test_split(df_fi['review_body'], true_labels, test_size=0.2, random_state=1)


In [27]:
# X_train_temp = X_train.apply(lambda x: wrd2vc_avg(x, google_wrd2vc))
# X_train_fnl = np.array(X_train_temp.values.tolist())

# X_test_temp= X_test.apply(lambda x: wrd2vc_avg(x, google_wrd2vc))
# X_test_fnl = np.array(X_test_temp.values.tolist())

# # Remove the NaN's (if any) and their labels from the training and test datasets 
# id_nan_trn = nan_id(X_train_fnl)
# if id_nan_trn == None:
#   Y_train_fnl = Y_train
# else:
#   X_train_fnl = np.delete(X_train_fnl, id_nan_trn, 0)
#   Y_train_fnl= np.delete(Y_train, id_nan_trn)

# '''make another Y_train_fnl for fnn so that dont have to run the whole thing again and dont make that variable as int since its not used ...so that ram is saved'''
# Y_trn_fnn_fnl=Y_train_fnl
# Y_train_fnl=Y_train_fnl.astype('int')

# id_nan_tst = nan_id(X_test_fnl)
# if id_nan_tst == None:
#   Y_test_fnl= Y_test
# else:
#   X_test_fnl = np.delete(X_test_fnl, id_nan_tst, 0)
#   Y_test_fnl = np.delete(Y_test, id_nan_tst)
# Y_tst_fnn_fnl=Y_test_fnl
# Y_test_fnl=Y_test_fnl.astype('int')

In [28]:
# clf=Perceptron()
# clf.fit(X_train_fnl,Y_train_fnl)

In [29]:
# predicted_perceptron=clf.predict(X_test_fnl)

In [30]:
#print(f'accuracry score:{accuracy_score(Y_test_fnl,predicted_perceptron)*100} %')

SVM Training

In [31]:
# clf_SVM=LinearSVC()
# clf_SVM.fit(X_train_fnl,Y_train_fnl)

In [32]:
#predicted_SVM=clf_SVM.predict(X_test_fnl)

In [33]:
#print(f'accruacy score for SVM is:{accuracy_score(Y_test_fnl,predicted_SVM)*100} %')

**FNN** 
Part 4a)

In [34]:
# true_labels= df_fi['star_rating'].to_numpy()
# X_trn_fnn, X_test_fnn, Y_trn_fnn, Y_test_fnn = train_test_split(df_fi['review_body'], true_labels, test_size=0.2, random_state=1)

In [35]:
#X_trn_fnn_fnl,Y_trn_fnn_fnl,X_tst_fnn_fnl,Y_tst_fnn_fnl=X_train_fnl,Y_trn_fnn_fnl,X_test_fnl,Y_test_fnl

In [36]:
#len(Y_trn_fnn_fnl)

In [13]:
class Train(Dataset):
    
    def __init__(self, xtrain, ytrain):
        'Initialization'
        self.data = xtrain
        self.labels = ytrain

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.data)

    def __getitem__(self, index):
        'Generates one sample of data'
        X = self.data[index]
        y = self.labels[index]

        return X, y

In [14]:
class Test(Dataset):
    def __init__(self, xtest, ytest):
        'Initialization'
        self.data = xtest
        self.labels = ytest

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.data)

    def __getitem__(self, index):
        'Generates one sample of data'
        X = self.data[index]
        y = self.labels[index]

        return X, y

In [39]:
# train_data_fnn, test_data_fnn = Train(X_trn_fnn_fnl, Y_trn_fnn_fnl-1), Test(X_tst_fnn_fnl, Y_tst_fnn_fnl-1)

In [40]:
# # number of subprocesses to use for data loading
# num_workers = 0
# # how many samples per batch to load
# batch_size = 100
# # percentage of training set to use as validation
# valid_size = 0.2

# # obtain training indices that will be used for validation
# num_train = len(train_data_fnn)
# indices = list(range(num_train))
# np.random.shuffle(indices)
# split = int(np.floor(valid_size * num_train))
# train_idx, valid_idx = indices[split:], indices[:split]

# # define samples for obtaining training and validation batches
# train_sampler = SubsetRandomSampler(train_idx)
# valid_sampler = SubsetRandomSampler(valid_idx)

# # prepare data loaders
# train_loader_fnn = torch.utils.data.DataLoader(train_data_fnn, batch_size=batch_size,
#                                            sampler=train_sampler)
# valid_loader_fnn = torch.utils.data.DataLoader(train_data_fnn, batch_size=batch_size,
#                                            sampler=valid_sampler)
# test_loader_fnn = torch.utils.data.DataLoader(test_data_fnn, batch_size=batch_size)

In [41]:
# # define the MLP architecture
# class ThreeLayerMLP(nn.Module):
#   def __init__(self, D_in, H1, H2, D_out):
#     super().__init__()
#     self.linear1 = nn.Linear(D_in, H1)
#     self.linear2 = nn.Linear(H1, H2)
#     self.linear3 = nn.Linear(H2, D_out)
#     # dropout layer (p=0.2)
#     self.dropout = nn.Dropout(0.2)

#   def forward(self, x):
#     # add hidden layer, with relu activation function
#     h1_relu = F.relu(self.linear1(x))
#     # add dropout layer
#     # add another hidden layer, with relu activation function
#     h2_relu = F.relu(self.linear2(h1_relu))
#     # add another dropout layer
#     h2_drop = self.dropout(h2_relu)
#     # add output layer
#     h2_output = self.linear3(h2_drop)

#     return h2_output

# # initialize MLP
# model_fnn = ThreeLayerMLP(300, 100, 10, 4)
# #model_fnn.cuda()
# print(model_fnn)

In [42]:
# # specify loss function (categorical cross-entropy)
# criterion = nn.CrossEntropyLoss()

# # specify optimizer (stochastic gradient descent) and learning rate = 0.01
# optimizer_fnn = torch.optim.SGD(model_fnn.parameters(), lr=0.01)

In [43]:
# import time

# start = time.time()
# # number of epochs to train the model
# n_epochs = 50

# # initialize tracker for minimum validation loss
# valid_loss_min = np.Inf # set initial "min" to infinity

# for epoch in range(n_epochs):
#   # monitor training loss
#   train_loss = 0.0
#   valid_loss = 0.0

#   ###################
#   # train the model #
#   ###################
#   model_fnn.train() # prep model for training
#   for data, target in train_loader_fnn:
#     # transfer data and target to GPU
#     data, target = data.to(device), target.to(device)
#     # clear the gradients of all optimized variables
#     optimizer_fnn.zero_grad()
#     # forward pass: compute predicted outputs by passing inputs to the model
#     output = model_fnn(data.float())
#     # calculate the loss
#     loss = criterion(output, target)
#     # backward pass: compute gradient of the loss with respect to model parameters
#     loss.backward()
#     # perform a single optimization step (parameter update)
#     optimizer_fnn.step()
#     # update running training loss
#     train_loss += loss.item()*data.size(0)

#   ######################
#   # validate the model #
#   ######################
#   model_fnn.eval() # prep model for evaluation
#   for data, target in valid_loader_fnn:
#     # transfer data and target to GPU
#     data, target = data.to(device), target.to(device)
#     # forward pass: compute predicted outputs by passing inputs to the model
#     output = model_fnn(data.float())
#     # calculate the loss
#     loss = criterion(output, target)
#     # update running validation loss
#     valid_loss += loss.item()*data.size(0)
    

#   # print training/validation statistics
#   # calculate average loss over an epoch
#   train_loss = train_loss/len(train_loader_fnn.dataset)
#   valid_loss = valid_loss/len(valid_loader_fnn.dataset)

#   print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
#         epoch+1, train_loss, valid_loss))
    
#   # save model if validation loss has decreased
#   if valid_loss <= valid_loss_min:
#     print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
#         .format(valid_loss_min, valid_loss))
#     torch.save(model_fnn.state_dict(), 'model.pt')
#     valid_loss_min = valid_loss

# end = time.time()
# print('Time elapsed: %.2f s' % (end - start))      

In [44]:
# Load the model with the lowest validation loss
#model_fnn.load_state_dict(torch.load('model.pt'))

In [45]:
# correct = 0
# total = 0
# model_fnn.eval()  # set the model to evaluation mode
# with torch.no_grad():
#     for embeddings, labels in test_loader_fnn:
#         # move data to the device
#         embeddings = embeddings.to(device)
#         labels = labels.to(device)
#         # calculate outputs by running embeddings through the network
#         outputs = model_fnn(embeddings.float())
#         # the class with the highest score is what we choose as prediction
#         _, predicted = torch.max(outputs, dim=1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# print('Accuracy_MLP-model_average-word-vectors_pretrained-word2vec-model_ternary-test-set: %d %%' % (100 * correct / total))

Part 4b)

In [46]:
## A function get the first 10 word vector embeddings and if length of reviews is less than 10 ..padding the remaining part to get to 10 length

In [47]:
# def concat_10_pad_10(sentence, model):
#     if type(sentence) == str:
#         sentence = sentence.split()

#     padded_vec = np.zeros((10, 300))

#     for i, word in enumerate(sentence[:10]):
#         if word in model:
#             padded_vec[i] = model[word]

#     return padded_vec.reshape(-1)



In [48]:
# true_labels= df_fi['star_rating'].to_numpy()
# X_trn_4b, X_test_4b, Y_trn_4b, Y_test_4b = train_test_split(df_fi['review_body'], true_labels, test_size=0.2, random_state=1)


In [49]:
# X_trn_4b_temp= X_trn_4b.apply(lambda x: concat_10_pad_10(x, google_wrd2vc))
# X_trn_4b_fnl = np.array(X_trn_4b_temp.values.tolist())

# X_tst_4b_temp= X_test_4b.apply(lambda x: concat_10_pad_10(x, google_wrd2vc))
# X_tst_4b_fnl = np.array(X_tst_4b_temp.values.tolist())

# # Remove the NaN's (if any) and their labels from the training and test datasets 
# id_nan_trn = nan_id(X_trn_4b_fnl)
# if id_nan_trn != None:
#   X_trn_4b_fnl = np.delete(X_trn_4b_fnl, id_nan_trn, 0)
#   Y_trn_4b_fnl= np.delete(Y_trn_4b, id_nan_trn)
# else:
#     Y_trn_4b_fnl = Y_trn_4b
# #Y_train_fnl=Y_train_fnl.astype('int')

# id_nan_tst = nan_id(X_tst_4b_fnl)
# if id_nan_tst != None:
#   X_tst_4b_fnl = np.delete(X_tst_4b_fnl, id_nan_tst, 0)
#   Y_tst_4b_fnl = np.delete(Y_test_4b, id_nan_tst)
# else:
#   Y_tst_4b_fnl=Y_test_4b
# #Y_test_fnl=Y_test_fnl.astype('int')

In [50]:
#train_data_fnn_10, test_data_fnn_10 = Train(X_trn_4b_fnl, Y_trn_4b_fnl-1), Test(X_tst_4b_fnl, Y_tst_4b_fnl-1)

In [51]:
# # number of subprocesses to use for data loading
# num_workers = 0
# # how many samples per batch to load
# batch_size = 100
# # percentage of training set to use as validation
# valid_size = 0.2

# # obtain training indices that will be used for validation
# num_train = len(train_data_fnn_10)
# indices = list(range(num_train))
# np.random.shuffle(indices)
# split = int(np.floor(valid_size * num_train))
# train_idx, valid_idx = indices[split:], indices[:split]

# # define samples for obtaining training and validation batches
# train_sampler = SubsetRandomSampler(train_idx)
# valid_sampler = SubsetRandomSampler(valid_idx)

# # prepare data loaders
# train_loader_fnn_10 = torch.utils.data.DataLoader(train_data_fnn_10, batch_size=batch_size,
#                                            sampler=train_sampler)
# valid_loader_fnn_10 = torch.utils.data.DataLoader(train_data_fnn_10, batch_size=batch_size,
#                                            sampler=valid_sampler)
# test_loader_fnn_10 = torch.utils.data.DataLoader(test_data_fnn_10, batch_size=batch_size)

In [52]:
#model_fnn_10 = ThreeLayerMLP(3000, 100, 10, 4)

In [53]:
# model_fnn_10 = ThreeLayerMLP(300, 100, 10, 4)
#print(model_fnn_10)

In [54]:
# # specify loss function (categorical cross-entropy)
# criterion = nn.CrossEntropyLoss()

# # specify optimizer (stochastic gradient descent) and learning rate = 0.01
# optimizer_fnn_10 = torch.optim.SGD(model_fnn_10.parameters(), lr=0.01)

In [55]:
# import time

# start = time.time()
# # number of epochs to train the model
# n_epochs = 20

# # initialize tracker for minimum validation loss
# valid_loss_min = np.Inf # set initial "min" to infinity

# for epoch in range(n_epochs):
#   # monitor training loss
#   train_loss = 0.0
#   valid_loss = 0.0

#   ###################
#   # train the model #
#   ###################
#   model_fnn_10.train() # prep model for training
#   for data, target in train_loader_fnn_10:
#     # transfer data and target to GPU
#     data, target = data.to(device), target.to(device)
#     # clear the gradients of all optimized variables
#     optimizer_fnn_10.zero_grad()
#     # forward pass: compute predicted outputs by passing inputs to the model
#     output = model_fnn_10(data.float())
#     # calculate the loss
#     loss = criterion(output, target)
#     # backward pass: compute gradient of the loss with respect to model parameters
#     loss.backward()
#     # perform a single optimization step (parameter update)
#     optimizer_fnn_10.step()
#     # update running training loss
#     train_loss += loss.item()*data.size(0)

#   ######################
#   # validate the model #
#   ######################
#   model_fnn_10.eval() # prep model for evaluation
#   for data, target in valid_loader_fnn_10:
#     # transfer data and target to GPU
#     data, target = data.to(device), target.to(device)
#     # forward pass: compute predicted outputs by passing inputs to the model
#     output = model_fnn_10(data.float())
#     # calculate the loss
#     loss = criterion(output, target)
#     # update running validation loss
#     valid_loss += loss.item()*data.size(0)
    

#   # print training/validation statistics
#   # calculate average loss over an epoch
#   train_loss = train_loss/len(train_loader_fnn_10.dataset)
#   valid_loss = valid_loss/len(valid_loader_fnn_10.dataset)

#   print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
#         epoch+1, train_loss, valid_loss))
    
#   # save model if validation loss has decreased
#   if valid_loss <= valid_loss_min:
#     print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
#         .format(valid_loss_min, valid_loss))
#     torch.save(model_fnn_10.state_dict(), 'model.pt')
#     valid_loss_min = valid_loss

# end = time.time()
# print('Time elapsed: %.2f s' % (end - start))      

In [56]:
# Load the model with the lowest validation loss
#model_fnn_10.load_state_dict(torch.load('model.pt'))

In [57]:
# correct = 0
# total = 0
# model_fnn_10.eval()  # set the model to evaluation mode
# with torch.no_grad():
#     for embeddings, labels in test_loader_fnn_10:
#         # move data to the device
#         embeddings = embeddings.to(device)
#         labels = labels.to(device)
#         # calculate outputs by running embeddings through the network
#         outputs = model_fnn_10(embeddings.float())
#         # the class with the highest score is what we choose as prediction
#         _, predicted = torch.max(outputs, dim=1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# print('Accuracy_MLP-model_average-word-vectors_pretrained-word2vec-model_ternary-test-set: %d %%' % (100 * correct / total))

**Part 5) Recurrent Neural Networks**

In [21]:
##Function to truncate longer reviews by 20 words and padding the smaller reviews to 20 to give input to the Functions of Recurrent Neural Network

# def trunc_20_pad_20(strings,model):
    
#       sent_split = [strings.split(' ') if isinstance(strings, str) else strings]

#        word_vector = []
#       for i in range(20):
            
#               try:
#                 wv = model[sent_split[i]]
#                 word_vector.append(wv)
#               except:
                
#                 pass

#               if len(word_vector) < 20:
#                 for _ in range(20-len(word_vector)):
#                   word_vector.append([0 for _ in range(300)])

#     return word_vector
def trunc_20_pad_20(strings,model):
    sent_split=[strings.split(' ') if isinstance (strings, str) else strings]
    
    wv=[]
    for i in range(20):
        try:
            temp=model[sent_split[i]]
            wv.append(temp)
        except:
            pass
        
    if len(wv)<20:
        for pad in range(20-len(wv)):
            wv.append([0 for pad in range(300)])
    
    return wv
        

In [22]:
true_labels= df_fi['star_rating'].to_numpy()
X_trn_RNN, X_test_RNN, Y_trn_RNN, Y_test_RNN = train_test_split(df_fi['review_body'], true_labels, test_size=0.2, random_state=1)

In [ ]:
X_trn_rnn_temp= X_trn_RNN.apply(lambda x: trunc_20_pad_20(x, google_wrd2vc))
X_trn_rnn_fnl = np.array(X_trn_rnn_temp.values.tolist())

X_tst_rnn_temp= X_test_RNN.apply(lambda x: trunc_20_pad_20(x, google_wrd2vc))
X_tst_rnn_fnl = np.array(X_tst_rnn_temp.values.tolist())

# Remove the NaN's (if any) and their labels from the training and test datasets 
# id_nan_trn = nan_id(X_trn_rnn_fnl)
# if id_nan_trn != None:
    
#     X_trn_rnn_fnl = np.delete(X_trn_rnn_fnl, id_nan_trn, 0)
#     Y_trn_rnn_fnl= np.delete(Y_trn_RNN, id_nan_trn)
# else:
Y_trn_rnn_fnl = Y_trn_RNN
# #Y_train_fnl=Y_train_fnl.astype('int')

# id_nan_tst = nan_id(X_tst_rnn_fnl)
# if id_nan_tst != None:
    
#     X_tst_rnn_fnl = np.delete(X_tst_rnn_fnl, id_nan_tst, 0)
#     Y_tst_rnn_fnl = np.delete(Y_test_RNN, id_nan_tst)
# else:
Y_tst_rnn_fnl=Y_test_RNN

In [26]:
#Creating the training and testing dataset

train_data_rnn= Train(X_trn_rnn_fnl, Y_trn_rnn_fnl- 1)
test_data_rnn = Test(X_tst_rnn_fnl, Y_tst_rnn_fnl - 1)

In [27]:
#Batching and Loading data for the RNN Model

num_workers = 0
batch_size = 16
valid_size = 0.2

num_train = len(train_data_rnn)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader_rnn = torch.utils.data.DataLoader(train_data_rnn, batch_size=batch_size,
                                           sampler=train_sampler, num_workers
                                           =num_workers)
valid_loader_rnn = torch.utils.data.DataLoader(train_data_rnn, batch_size=batch_size,
                                           sampler=valid_sampler, num_workers
                                           =num_workers)
test_loader_rnn = torch.utils.data.DataLoader(test_data_rnn, batch_size=batch_size,
                                           num_workers=num_workers)

In [28]:
class RNN(nn.Module):
    
      def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()
        # Number of hidden dimensions
        self.hidden_dim = hidden_dim
        # Number of hidden layers
        self.layer_dim = layer_dim
        # RNN
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True,
                         nonlinearity='relu')
        # Output layer
        self.fc = nn.Linear(hidden_dim, output_dim)
    
      def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(device)
        # One time step
        out, hn = self.rnn(x, h0)
        out=out.contiguous().view(-1,out.shape[1]*out.shape[2])
        out = self.fc(out)
        return out

In [29]:
model_rnn_nrml=RNN(300,20,1,4)
#model_rnn_nrml.cuda()
print(model_rnn_nrml)

RNN(
  (rnn): RNN(300, 20, batch_first=True)
  (fc): Linear(in_features=20, out_features=4, bias=True)
)


In [30]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01

optimizer_rnn = torch.optim.SGD(model_rnn_nrml.parameters(), lr=0.01)

In [31]:
import time

start = time.time()
# number of epochs to train the model
n_epochs = 20

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
  # monitor training loss
  train_loss = 0.0
  valid_loss = 0.0

  ###################
  # train the model #
  ###################
  model_rnn_nrml.train() # prep model for training
  for data, target in train_loader_rnn:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # clear the gradients of all optimized variables
    optimizer_rnn.zero_grad()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_rnn_nrml(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer_rnn.step()
    # update running training loss
    train_loss += loss.item()*data.size(0)

  ######################
  # validate the model #
  ######################
  model_rnn_nrml.eval() # prep model for evaluation
  for data, target in valid_loader_rnn:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_rnn_nrml(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # update running validation loss
    valid_loss += loss.item()*data.size(0)
    

  # print training/validation statistics
  # calculate average loss over an epoch
  train_loss = train_loss/len(train_loader_rnn.dataset)
  valid_loss = valid_loss/len(valid_loader_rnn.dataset)

  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, train_loss, valid_loss))
    
  # save model if validation loss has decreased
  if valid_loss <= valid_loss_min:
    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
        .format(valid_loss_min, valid_loss))
    torch.save(model_rnn_nrml.state_dict(), 'model.pt')
    valid_loss_min = valid_loss

end = time.time()
print('Time elapsed: %.2f s' % (end - start))

      

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found object

In [ ]:
# Load the model with the lowest validation loss
model_rnn_nrml.load_state_dict(torch.load('model.pt'))

In [ ]:

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for out outputs
with torch.no_grad():
  for data in test_loader_rnn:
    embeddings, labels = data
    # transfer data and target to GPU
    embeddings, labels = embeddings.to(device), labels.to(device)
    # calculating outputs by running embeddings through the network
    model_rnn_nrml.to(device)
    outputs = model_rnn_nrml(embeddings.float())
    # the class with the highest score is what we choose as prediction
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy_simple-RNN-model_pretrained-word2vec_ternary-test-set: %d %%' % (100 * correct / total))

**Part 5b a Gated Recurrent Unit (GRU) cell**

In [ ]:
train_data_GRU=train_data_rnn
test_data_GRU=test_data_rnn

In [ ]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 100
# percentage of training set to use as validation
valid_size = 0.2

# obtain training indices that will be used for validation
num_train = len(train_data_GRU)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samples for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders
train_loader_GRU = torch.utils.data.DataLoader(train_data_GRU, batch_size=batch_size,
                                           sampler=train_sampler, num_workers
                                           =num_workers)
valid_loader_GRU = torch.utils.data.DataLoader(train_data_GRU, batch_size=batch_size,
                                           sampler=valid_sampler, num_workers
                                           =num_workers)
test_loader_GRU= torch.utils.data.DataLoader(test_data_GRU, batch_size=batch_size,
                                           num_workers=num_workers)

In [ ]:
# define the GRU architecture
class GRU(nn.Module):
  def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
    super().__init__()
    # Number of hidden dimensions
    self.hidden_dim = hidden_dim
    # Number of hidden layers
    self.layer_dim = layer_dim
    # GRU
    self.gru = nn.GRU(input_dim, hidden_dim, layer_dim, batch_first=True)
    # Output layer
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, x):
    # Initialize hidden state with zeros
    h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(device)
    # One time step
    out, hn = self.gru(x, h0)
    out = self.fc(out[:, -1, :])
    return out

In [ ]:
model_GRU = GRU(300, 20, 1, 5)
print(model_GRU)

In [ ]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01

optimizer_GRU = torch.optim.SGD(model_GRU.parameters(), lr=0.01)

In [ ]:
import time

start = time.time()
# number of epochs to train the model
n_epochs = 20

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
  # monitor training loss
  train_loss = 0.0
  valid_loss = 0.0

  ###################
  # train the model #
  ###################
  model_GRU.train() # prep model for training
  for data, target in train_loader_GRU:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # clear the gradients of all optimized variables
    optimizer_GRU.zero_grad()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_GRU(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer_GRU.step()
    # update running training loss
    train_loss += loss.item()*data.size(0)

  ######################
  # validate the model #
  ######################
  model_GRU.eval() # prep model for evaluation
  for data, target in valid_loader_GRU:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_GRU(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # update running validation loss
    valid_loss += loss.item()*data.size(0)
    

  # print training/validation statistics
  # calculate average loss over an epoch
  train_loss = train_loss/len(train_loader_GRU.dataset)
  valid_loss = valid_loss/len(valid_loader_GRU.dataset)

  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, train_loss, valid_loss))
    
  # save model if validation loss has decreased
  if valid_loss <= valid_loss_min:
    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
        .format(valid_loss_min, valid_loss))
    torch.save(model_GRU.state_dict(), 'model.pt')
    valid_loss_min = valid_loss

end = time.time()
print('Time elapsed: %.2f s' % (end - start))

      

In [ ]:
# Load the model with the lowest validation loss
model_GRU.load_state_dict(torch.load('model.pt'))

In [ ]:

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for out outputs
with torch.no_grad():
  for data in test_loader_GRU:
    embeddings, labels = data
    # transfer data and target to GPU
    embeddings, labels = embeddings.to(device), labels.to(device)
    # calculating outputs by running embeddings through the network
    model_GRU.to(device)
    outputs = model_GRU(embeddings.float())
    # the class with the highest score is what we choose as prediction
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy_simple-RNN-model_pretrained-word2vec_ternary-test-set: %d %%' % (100 * correct / total))

**Part 5c) LSTM network **

In [ ]:
train_data_LSTM=train_data_rnn
test_data_LSTM=test_data_rnn

In [ ]:
# class LSTM(nn.Module):
#     def __init__(self, input_size, output_size, hidden_dim, n_layers, model_type = "lstm"):
#         super(LSTM, self).__init__()
#         self.hidden_dim = hidden_dim
#         self.n_layers = n_layers
#         self.model_type = model_type
#         self.layer = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
#         self.fc = nn.Linear(400, output_size)

#     def forward(self, x):
#         batch_size = x.size(0)
#         hidden = self.init_hidden(batch_size)
#         out, hidden = self.layer(x, hidden)
#         out = out.contiguous().view(-1, out.shape[1] * out.shape[2])
#         out = self.fc(out)
#         return out, hidden

#     def init_hidden(self, batch_size):
#         hidden = (torch.zeros(self.n_layers, batch_size, self.hidden_dim),torch.zeros(self.n_layers, batch_size, self.hidden_dim))
#         return hidden

class LSTM(nn.Module):
  def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
    super().__init__()
    # Number of hidden dimensions
    self.hidden_dim = hidden_dim
    # Number of hidden layers
    self.layer_dim = layer_dim
    # GRU
    self.LSTM = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
    # Output layer
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, x):
    # Initialize hidden state with zeros
   
    h0 =  (torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(device), torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(device))
    # One time step
    out, hn = self.LSTM(x, h0)
    out = self.fc(out[:, -1, :])
    return out

In [ ]:
model_LSTM = LSTM(300, 20, 1, 4)
print(model_LSTM)

In [ ]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 100
# percentage of training set to use as validation
valid_size = 0.2

# obtain training indices that will be used for validation
num_train = len(train_data_LSTM)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samples for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders
train_loader_LSTM = torch.utils.data.DataLoader(train_data_LSTM, batch_size=batch_size,
                                           sampler=train_sampler, num_workers
                                           =num_workers)
valid_loader_LSTM = torch.utils.data.DataLoader(train_data_LSTM, batch_size=batch_size,
                                           sampler=valid_sampler, num_workers
                                           =num_workers)
test_loader_LSTM= torch.utils.data.DataLoader(test_data_LSTM, batch_size=batch_size,
                                           num_workers=num_workers)

In [ ]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01

optimizer_LSTM = torch.optim.SGD(model_LSTM.parameters(), lr=0.01)

In [ ]:
import time

start = time.time()
# number of epochs to train the model
n_epochs = 20

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
  # monitor training loss
  train_loss = 0.0
  valid_loss = 0.0

  ###################
  # train the model #
  ###################
  model_LSTM.train() # prep model for training
  for data, target in train_loader_LSTM:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # clear the gradients of all optimized variables
    optimizer_LSTM.zero_grad()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_LSTM(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer_LSTM.step()
    # update running training loss
    train_loss += loss.item()*data.size(0)

  ######################
  # validate the model #
  ######################
  model_LSTM.eval() # prep model for evaluation
  for data, target in valid_loader_LSTM:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_LSTM(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # update running validation loss
    valid_loss += loss.item()*data.size(0)
    

  # print training/validation statistics
  # calculate average loss over an epoch
  train_loss = train_loss/len(train_loader_LSTM.dataset)
  valid_loss = valid_loss/len(valid_loader_LSTM.dataset)

  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, train_loss, valid_loss))
    
  # save model if validation loss has decreased
  if valid_loss <= valid_loss_min:
    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
        .format(valid_loss_min, valid_loss))
    torch.save(model_LSTM.state_dict(), 'model.pt')
    valid_loss_min = valid_loss

end = time.time()
print('Time elapsed: %.2f s' % (end - start))

      

In [ ]:
# Load the model with the lowest validation loss
model_LSTM.load_state_dict(torch.load('model.pt'))

In [ ]:

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for out outputs
with torch.no_grad():
  for data in test_loader_LSTM:
    embeddings, labels = data
    # transfer data and target to GPU
    embeddings, labels = embeddings.to(device), labels.to(device)
    # calculating outputs by running embeddings through the network
    model_LSTM.to(device)
    outputs = model_LSTM(embeddings.float())
    # the class with the highest score is what we choose as prediction
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy_simple-LSTM-model_pretrained-word2vec_ternary-test-set: %d %%' % (100 * correct / total))